In [1]:
import pandas as pd
import numpy as np
import requests

1. A dataset containing details about Metro Nashville Police Department reported incidents is available at https://data.nashville.gov/Police/Metro-Nashville-Police-Department-Incidents/2u6v-ujjs. 

Make use of the API to find all aggravated burglary incidents that were reported during the six month period from January 1, 2021 through June 30, 2021.

In [25]:
endpoint = 'https://data.nashville.gov/resource/2u6v-ujjs.json'

In [88]:
params = {
    'offense_description': 'BURGLARY',
    'location_description': 'RESIDENCE, HOME',
    '$where': 'incident_reported between \'2021-01-01\' AND \'2021-06-30\''
}

In [89]:
response = requests.get(endpoint,
                        params = params )

In [90]:
response

<Response [200]>

In [91]:
response.json()

[{'primary_key': '20210127959_11',
  'incident_number': '20210127959',
  'report_type': 'D',
  'report_type_description': 'DISPATCHED',
  'incident_status_code': 'O',
  'incident_status_description': 'OPEN',
  'investigation_status': 'Open',
  'incident_occurred': '2021-03-04T11:00:00.000',
  'incident_reported': '2021-03-04T13:48:00.000',
  'incident_location': 'CEDARMONT DR',
  'latitude': '36.04',
  'longitude': '-86.7',
  'location_code': '22',
  'location_description': 'RESIDENCE, HOME',
  'offense_number': '1',
  'offense_nibrs': '220',
  'offense_description': 'BURGLARY',
  'weapon_primary': '17',
  'weapon_description': 'NONE',
  'victim_number': '1',
  'domestic_related': False,
  'victim_type': 'I',
  'victim_description': 'INDIVIDUAL (18 AND OVER)',
  'victim_gender': 'F',
  'victim_race': 'W',
  'victim_ethnicity': 'Hispanic',
  'victim_county_resident': 'RESIDENT',
  'mapped_location': {'type': 'Point', 'coordinates': [-86.7, 36.04]}},
 {'primary_key': '20210125875_11',
  